# NMR Processing Overview

---

1. Split files into different categories.
    1. How many individual fids?
    2. How many array experiments?
    3. How are temperature sets stored?
    4. How are materials stored?
2. Develop / confirm metadata for those categories.
    + Cross reference with documentation provided by Trent.
    + Compare processing demo results to Trent's data. 
    + Meet with Trent to confirm assignments.
3. Prioritize subsets.
3. **Design Bokeh application**
4. Process subsets.

#### Set Local Data Path

---

In [1]:
data_folder = '/home/tyler/data/Sep-2016-23Na'

#### Import Packages

---

In [7]:
import nmrglue as ng
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
%matplotlib inline

In [3]:
from trentnmr import *

## Splitting Files into Categories

Using glob to get all the folders in the data directory.

---

In [9]:
array_glob = data_folder + '/**/*arrays*.fid'
all_fid_glob = data_folder + '/**/*.fid'

array_folders = {fn for fn in glob.iglob(array_glob, recursive=True)}
all_fid_folders = {fn for fn in glob.iglob(all_fid_glob, recursive=True)}

fid_folders = all_fid_folders - array_folders

#### How many individual fids are there?

#### How many array files are there?

In [71]:
print('All fid folders: ', len(all_fid_folders))
print('Array folders: ', len(array_folders))
print('Fid folders: ', len(fid_folders))

All fid folders:  301
Array folders:  86
Fid folders:  215


#### How are temperatures stored?

Build some usefull regex strings.

In [102]:
deg_C_regx = "([0-9]*)C"
all_hz_glob = "([0-9])+(Hz)"
molarity_glob = "([0-9]*(p)*([0-9])+(M)-(AlOH3|LiOH|NaOH))+"
nmr_element_glob = "(27Al|23Na)"
gibbsite_glob = "(\w*-)(Gibbsite)"
milli_gram_glob = "(\d*)p(\d*)mg"

In [98]:
# for file in all_fid_folders:
#     x = re.findall(deg_C_regx, file)
#     if x: print(x, file)
#     else: print("NO TEMPERATURE FOUND",file)

#### How are Materials stored?

**Gibbsite**

In [106]:
gibbsite_arrays = {x if re.search(gibbsite_glob, x) else None for x in all_fid_folders}
print(*filter(None, gibbsite_arrays))

/home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/23Na-concentrated-Gibbsite-2755Hz-90C-32ctF.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/27Al-concentrated-Gibbsite-2767Hz-170C-22arrays-128ctF.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/23Na-concentrated-Gibbsite-2770Hz-70C-32ct.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/27Al-concentrated-Gibbsite-2735Hz-130C-128ctF.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/27Al-concentrated-Gibbsite-2782Hz-50C-128ctF.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/23Na-concentrated-Gibbsite-2782Hz-50C-32ct.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/27Al-concentrated-Gibbsite-2755Hz-90C-128ct.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/23Na-concentrated-Gibbsite-2744Hz-110C-32ctF.fid /home/tyler/data/Sep-2016-2

**Molarity**

In [107]:
# for file in all_fid_folders:
#     x = re.findall(molarity_glob, file)
#     if x: print(x, file)
#     else: print("NO MOLARITY FOUND",file)

#### Milligram Weights

In [108]:
milligrams = {x if re.search(milli_gram_glob, x) else None for x in all_fid_folders}
# print(*filter(None, gibbsite_arrays))

# for file in all_fid_folders:
#     x = re.findall(deg_C_regx, file)
#     if x: print(x, file)
#     else: print("NO TEMPERATURE FOUND",file)

/home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/23Na-concentrated-Gibbsite-2755Hz-90C-32ctF.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/27Al-concentrated-Gibbsite-2767Hz-170C-22arrays-128ctF.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/23Na-concentrated-Gibbsite-2770Hz-70C-32ct.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/27Al-concentrated-Gibbsite-2735Hz-130C-128ctF.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/27Al-concentrated-Gibbsite-2782Hz-50C-128ctF.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/23Na-concentrated-Gibbsite-2782Hz-50C-32ct.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/27Al-concentrated-Gibbsite-2755Hz-90C-128ct.fid /home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/23Na-concentrated-Gibbsite-2744Hz-110C-32ctF.fid /home/tyler/data/Sep-2016-2